## Часть 1 Первоначальное ознакомление с данными

In [145]:
import pandas as pd

import warnings

warnings.filterwarnings(action='ignore')

In [146]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [147]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Изучим общую информацию**

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Часть 2 Предобработка данных

**Проверим наличие пропусков**

In [149]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце приоритетнее медианным значением по каждому типу из столбца `income_type`.**

In [150]:
for p in data['income_type'].unique():
    data.loc[(data['income_type'] == p) & (data['total_income'].isna()), 'total_income'] = \
data.loc[(data['income_type'] == p), 'total_income'].median()




**Обнаружен артефакт в данных отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [151]:
data['days_employed'] = abs(data['days_employed'])

**Проверим медианное значение трудового стажа для каждого типа занятовсти в днях.**

In [152]:
median_value = data.groupby('income_type')['days_employed'].median()
print(median_value)

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


**У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.**

**Выведем перечень уникальных значений столбца `children`.**

In [153]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [154]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Проверим изменения**

In [155]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [156]:
for p in data['income_type'].unique():
    data.loc[(data['income_type'] == p) & (data['days_employed'].isna()), 'days_employed'] = \
data.loc[(data['income_type'] == p), 'days_employed'].median()

**Убедимся, что все пропуски заполнены.**

In [157]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Изменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [158]:
data['total_income'] = data['total_income'].astype('int')

**Избавимся от неявных дубликатов в столбце `education`. В этом столбце одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.**

In [159]:
data['education'] = data['education'].str.lower()

**Проверим и удалим имеющиеся явные дубликаты.**

In [160]:
data.duplicated().sum()

71

In [161]:
data = data.drop_duplicates()

## Часть 3 Категоризация данных

**Создадим категории заёмщиков на основании стобца с доходами `total_income_category`:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [162]:
def categorize_income (income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'

In [163]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Посмотрим на перечень уникальных целей взятия кредита из столбца `purpose`.**

In [164]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [165]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [166]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

print(data)

       children  days_employed  dob_years education  education_id  \
0             1    8437.673028         42    высшее             0   
1             1    4024.803754         36   среднее             1   
2             0    5623.422610         33   среднее             1   
3             3    4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1    4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1    2113.346888         38   среднее             1   
21523         3    3112.481705         38   среднее             1   
21524         2    1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

**Столбец с целью кредита успешно добавлен в таблицу**

## Часть 4 Исследовательский анализ данных

**Проверим есть ли зависимость между количеством детей и возвратом кредита в срок**

In [172]:
def children_group(child):
    if child >= 3:
        return 'Многодетные'
    if child == 0:
        return 'Бездетные'
    if child == 1:
        return 'Люди с одним ребёнком'
    if child == 2:
        return 'Люди с двумя детьми'
    return 'Больше 6 детей'

data['children_group'] = data['children'].apply(children_group)

data_children = data.pivot_table(index=['children_group'], values='debt', aggfunc= ('sum', 'count')).reset_index()

data_children['proportion'] = round(data_children['sum']/data_children['count'], 3) * 100

data_children

,children_group,count,sum,proportion
0,Бездетные,14091,1063,7.5
1,Люди с двумя детьми,2052,194,9.5
2,Люди с одним ребёнком,4808,444,9.2
3,Многодетные,380,31,8.2


В данной таблице мы объединили в одну группу людей с тремя и более детьми в категорию 'Многодетные', так как их количество значительно меньше чем людей других категорий и процент их надёжности к кредитованию мог бы привести исследование к неправильным выводам. Сводная таблица показывает что между потенциальными заёмщиками без детей и остальными категориями есть только незначительная разница в проценте должников по кредитам. Вероятно, дело в том что люди не имеющие детей имеют меньше обстоятельств, которые мешают им вносить платежи по кредиту вовремя. Данный показатель принесёт мало пользы в построении скоринговой модели.

**Проверим есть ли зависимость между семейным положением и возвратом кредита в срок**

In [173]:
data_family = data.pivot_table(index=['family_status'], values='debt', aggfunc=('sum','count'))

data_family ['proportion'] = round(data_family['sum']/data_family['count'], 3) * 100

data_family = data_family.sort_values(by='proportion',ascending=True)

print(data_family)

                       count  sum  proportion
family_status                                
вдовец / вдова           951   63         6.6
в разводе               1189   84         7.1
женат / замужем        12261  927         7.6
гражданский брак        4134  385         9.3
Не женат / не замужем   2796  273         9.8


 В данной таблице мы объединили в одну группу людей с тремя и более детьми в категорию 'Многодетные', так как их количество значительно меньше чем людей других категорий и процент их надёжности к кредитованию мог бы привести исследование к неправильным выводам. Сводная таблица показывает что между потенциальными заёмщиками без детей и остальными категориями есть только незначительная разница в проценте должников по кредитам. Вероятно, дело в том что люди не имеющие детей имеют меньше обстоятельств, которые мешают им вносить платежи по кредиту вовремя. Данный показатель принесёт мало пользы в построении скоринговой модели

**Проверим есть ли зависимость между уровнем дохода и возвратом кредита в срок**

In [177]:
def income_group(income):
    if 0 <= income <= 100000:
        return 'До 100000'
    if 0 <= income <= 150000:
        return 'От 100000 до 150000'
    if 150000 <= income <= 2000000:
        return 'От 150000 до 2000000'

data['income_group'] = data['total_income'].apply(income_group)

data_income = data.pivot_table(index=['income_group'], values='debt', aggfunc=['sum','count'])

data_income ['proportion'] = round(data_income['sum']/data_income['count'], 4) * 100

data_income = data_income.sort_values(by='proportion',ascending=True)

print(data_income)

                      sum count proportion
                     debt  debt           
income_group                              
От 150000 до 2000000  758  9775       7.75
До 100000             354  4444       7.97
От 100000 до 150000   619  7110       8.71


Вывод: В данной таблице мы разбили заёмщиков на три категории исходя из доходов, так как категория людей с доходом до 50000 тысяч рублей включала в себя небольшое количество людей. Таблица показывает что самым ненадёжным классом заёмщиков являются люди из категорит 'от 100000 до 150000'. А самыми надёжными считаются люди из категории 'от 150000 до 2000000'. Предположение №1: Вероятно, такое соотношение вызвано тем что люди из категории 'до 100000' берут кредиты на менее весомые суммы и вовремя их отдают. Предположение №2:Люди категории 'до 100000', не так сильно загружены как люди из других двух категорий, поэтому не забывают о своевременном платеже по кредиту. Предположение №3: Люди категории 'От 150000 до 2000000' меньше рискуют попасть в ситуацию, когда нет средств для внесения платежа. Можно было бы предположить что категория '100000 до 150000' является ненадёжной категорией заёмщиков, но разница между категориями очень мала. Соответственно, только по этому критерию мы тоже не можем полноценно оценивать заёмщика.

In [182]:
data_purpose = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=('sum','count'))

data_purpose ['proportion'] = round(data_purpose['sum']/data_purpose['count'], 4) * 100

data_purpose

,count,sum,proportion
purpose_category,,,
операции с автомобилем,4279,400,9.35
операции с недвижимостью,10751,780,7.26
получение образования,3988,369,9.25
проведение свадьбы,2313,183,7.91


Вывод: Все цели кредита разбиты на 4 основные категории. Исходя из данных таблицы видно что есть различие по возврату задолженности в срок - менее рискована выдача кредитов из категории 'операции с недвижимостью' и проведение свадьбы'. Более рискована 'получение образования' и 'операции с автомобилем'. Разница в проценте просрочивших выплату по кредиту не сильно разнятся, поэтому этот критерий тоже одиночно не показывает реальную разницу между заёмщиками из данных категорий. Предположение: Проведение свадьбы - единоразовое событие и в ценовом сегменте самое бюджетное из представленного. Приобретение недвижимости - достаточно серьёзное решение - поэтому и выплатыы производятся регулярно. В то же время, получение образования и приобретение автомобиля - зачастую мероприятие для людей юного возраста и данная категория людей может быть достаточно легкомысленна.

## Часть 5 Общие итоги исследования

Нашими усилиями было проделано исследование с целью проверки влияния

различных факторов на возврат кредита в сроки установленные договором.

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного

скоринга — специальной системы, которая оценивает способность потенциального

заёмщика вернуть кредит банку.

   **В данном иследовании перед нами стояла задача проверить 4 гипотезы:**

1.Количество детей потенциального кредитополучателя влияет на возврат кредита в срок.

2.Семейное положение потенциального кредитополучателявлияет на возврат кредита в срок.

3.Общий доход потенциального кредитополучателя влияет на возврат кредита в срок.

4.Назначение кредита потенциального кредитополучателя влияет на возврат кредита в срок.

В ходе проверки гепотез мы получили следующие результаты:

**Первая гипотеза частично подтверждена:**

Процент просроченных кредитов, среди кредитополучателей **не имеющих детей** - **7.6%**

Процент просроченных кредитов, среди **многодетных кредитополучателей** - **8.1%**

Процент просроченных кредитов, среди кредитополучателей **имеющих одного ребёнка** - **9.2%**

Процент просроченных кредитов, среди кредитополучателей **имеющих двух детей** - **9.4%**

**Рекомендуется:** Увеличить выборку по всем четырём категориям, в особенности по многодетной категории заёмщиков. Вследствие увеличения выборки можно будет отследить тенденцию невозврата кредита в срок среди разных категорий многодетных заёмщиков.

**Вторая гипотеза подтверждена:**

Процент просроченных кредитов, среди кредитополучателей из категории **"вдовец/вдова"** - **6.7%**

Процент просроченных кредитов, среди кредитополучателей из категории **"в разводе "** - **7.0%**

Процент просроченных кредитов, среди кредитополучателей из категории **"женат / замужем "** - **7.6%**          
Процент просроченных кредитов, среди кредитополучателей из категории **"гражданский брак"** - **9.5%**

Процент просроченных кредитов, среди кредитополучателей из категории **"не женат/не замужем"** - **9.7%**

**Рекомендуется:** Увеличить выборку для всех категорий, есть вероятность, что новые данные изменят картину и какая-то из категорий покажет аномальные результаты. До увеличения выборки не рекомендуется данный критерий как причину в отказе в выдаче кредита, либо назначению повышенного процента кредитной ставки.

**Третья гипотеза частично подтверждена:**

Процент просроченных кредитов, среди кредитополучателей из категории с доходом **"от 150 до 200 т.р."** - **7.8%**   

Процент просроченных кредитов, среди кредитополучателей из категории с доходом **"до 100 т.р."** -     **8.0%**   

Процент просроченных кредитов, среди кредитополучателей из категории с доходом **"от 100 до 150 т.р."** - **8.8%**
**Рекомендуется:** Слишком малая выборка данных не дала возможность для полного подтверждения или опровержения данной гипотезы. Исходя из данного результата - заработок нельзя считать весомым критерием при рассмотрении потенциального заёмщика.При возможности уточнить реальный зароботок клиентов с пропущенными значениями, впоследствии заполненными медианным значением.

**Четвёртая гипотеза подтверждена:**
Процент просроченных кредитов, среди кредитополучателей с целью получения кредита связанным с **операциями с недвижимостью** - **7.2%**    

Процент просроченных кредитов, среди кредитополучателей с целью получения кредита связанным с **проведением свадьбы** - **8.0%**   

Процент просроченных кредитов, среди кредитополучателей с целью получения кредита связанным с **получением образования** - **9.2%**    

Процент просроченных кредитов, среди кредитополучателей с целью получения кредита связанным с **операциями с автомобилем** - **9.3%**    

**Рекомендуется:** Увеличение выборки среди клиентов. Так как разница в процентном отношении между всеми категориями цели не слишком высока, рекомендуется не придавать веское значение данному критерию.

**Необходимые действия:**

1.Увеличение выборки во всех четырёх критериях (в особенности в первом и третьем). Второй и четвёртый уже готовы к использованию, но наилучшим решением будет расширение их выборки.

2.Повторный анализ с учётом поступивших данных.

3.Построение скоринговой кредитной модели с учётом проведённого исследования.


